# Rental Price Prediction


In this project we will try to predict the price of rental apartments in Berlin by extracting data via Selenium

In [2]:
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException 
import time
from geopy.geocoders import Nominatim
from googletrans import Translator

import winsound
import pywhatkit


# Crawling

At this step we go for each page and take the urls of the uloaded apartments

In [21]:
def get_urls(page_number):
    page_number = str(page_number)
    web = 'https://www.wg-gesucht.de/1-zimmer-wohnungen-und-wohnungen-in-Berlin.8.1+2.1.'+page_number+'.html?pagination=1&pu='
    PATH = r'C:\Program Files (x86)\chromedriver.exe'
    driver_url = webdriver.Chrome(PATH)
    driver_url.get(web)
    time.sleep(10)
    try:
        driver.find_element_by_xpath("//a[@class='cmpboxbtn cmpboxbtnyes']").click()
    except:
        pass

     #capcha alret
    try:
        WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR,"iframe[name^='a-'][src^='https://www.google.com/recaptcha/api2/anchor?']")))
        #WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//span[@id='recaptcha-anchor']"))).click()
        print('capcha')
        winsound.Beep(1000, 500)
        winsound.Beep(1000, 500)
        winsound.Beep(1000, 500)
        pywhatkit.sendwhatmsg('+9720545488595', 'Capcha', int(time.ctime()[11:13]), int(time.ctime()[14:16]) + 2 ,tab_close=False)
        input("Press Enter to continue...")
    except:
        pass
    
    
    urls = []
    url_list = driver_url.find_elements_by_xpath('//h3[@class="truncate_title noprint"]/a')

    for row in url_list:
        urls.append(row.get_attribute('href'))
    driver_url.quit()
    return urls

here we get all the parameters for the urls that we took
we are taking data from German site so we use Translator() to translate to English

also the site have Capcha v2 which you need to solve it by pressing the right pictures, so if there would be a Capcha it will use winsound.Beep which will beep 3 times and also send me Whatsup message with pywhatkit.sendwhatmsg if im not next to the computer

I learned Selenium from two youtubers : 
1) https://www.youtube.com/watch?v=lTypMlVBFM4&t=245s&ab_channel=JohnWatsonRooney

2) https://www.youtube.com/watch?v=Xjv1sY630Uc&ab_channel=TechWithTim

In [4]:
apartment_url = []
size = []
price = []
numOfRooms = []
buildingCond = []
floor = []
hasKitchen = []
heatingType = []
longitude = []
latitude = []
bus_walk = []
parking = []


#folder:
hasWashingMachine = []
hasBalcony = []
hasElevetor = []
###

geolocator = Nominatim(user_agent='myapplication')
translator = Translator()


PATH = r'C:\Program Files (x86)\chromedriver.exe'
driver = webdriver.Chrome(PATH)

for page in range(0,100):
    print(f'page : {page+1}')
    urls = get_urls(page)
    for url in urls:
        if "sort_order" in url: 
            continue
        print(url)
        apartment_url.append(url)
        driver.get(url)
        time.sleep(5)
        
        #accept cookies
        try:
            driver.find_element_by_xpath("//a[@class='cmpboxbtn cmpboxbtnyes']").click()
        except Exception:
            pass
        
        
        #capcha alret
        try:
            WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR,"iframe[name^='a-'][src^='https://www.google.com/recaptcha/api2/anchor?']")))
            #WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//span[@id='recaptcha-anchor']"))).click()
            print('capcha')
            winsound.Beep(1000, 500)
            winsound.Beep(1000, 500)
            winsound.Beep(1000, 500)
            pywhatkit.sendwhatmsg('+9720545488595', 'Capcha', int(time.ctime()[11:13]), int(time.ctime()[14:16]) + 2 ,tab_close=False)
            input("Press Enter to continue...")    
        except:
            pass 
            

        try:
            size.append(driver.find_elements_by_xpath("//label[@class='amount']")[0].text)
            numOfRooms.append(driver.find_elements_by_xpath("//label[@class='amount']")[1].text)
            price.append(driver.find_elements_by_xpath("//label[@class='amount']")[2].text)
        except NoSuchElementException:
            size.append(None)
            numOfRooms.append(None)
            price.append(None)
     


    #floor
        try:
            child= driver.find_element_by_xpath("//span[@class='mdi mdi-office-building mdi-36px noprint']")
            parent= child.find_element_by_xpath("..")
            translated = translator.translate(parent.get_attribute("innerText").split('\n')[1])
            floor.append(translated.text)
        except NoSuchElementException:
            floor.append(None)
    #kitchen        
        try:
            child= driver.find_element_by_xpath("//span[@class='mdi mdi-silverware-fork-knife mdi-36px noprint']")
            parent= child.find_element_by_xpath("..")
            translated = translator.translate(parent.get_attribute("innerText").split('\n')[1])
            hasKitchen.append(translated.text)
        except NoSuchElementException:
            hasKitchen.append(None)

    #heating        
        try:
            child= driver.find_element_by_xpath("//span[@class='mdi mdi-fire mdi-36px noprint']")
            parent= child.find_element_by_xpath("..")
            translated = translator.translate(parent.get_attribute("innerText").split("\n")[1])
            heatingType.append(translated.text)
        except NoSuchElementException:
            heatingType.append(None)  


    #folder:
        try:
            child= driver.find_element_by_xpath("//span[@class='mdi mdi-folder mdi-36px noprint']")
            parent= child.find_element_by_xpath("..")
            folder = parent.get_attribute("innerText").split("\n")[1]
            if 'Waschmaschine' in folder:
                hasWashingMachine.append(1)
            else:
                hasWashingMachine.append(0)
            if 'Balkon' in folder:
                hasBalcony.append(1)
            else:
                hasBalcony.append(0)
            if 'Aufzug' in folder:
                hasElevetor.append(1)
            else:
                hasElevetor.append(0)
        except NoSuchElementException:
            print("no folder")
            hasWashingMachine.append(0)
            hasBalcony.append(0)
            hasElevetor.append(0)

    #location:
        try:
            adress = driver.find_element_by_xpath("//div[@class='col-sm-4 mb10']/a")
            adress = adress.get_attribute("innerText").replace('\n',' ')
            location = geolocator.geocode(adress)
            longitude.append(location.longitude)
            latitude.append(location.latitude)
        except:
            longitude.append(None)
            latitude.append(None)
            
            
            
    #bus walk:
        try:
            child= driver.find_element_by_xpath("//span[@class='mdi mdi-bus mdi-36px noprint']")
            parent= child.find_element_by_xpath("..")
            bus_walk.append(int(parent.get_attribute("innerText").split('\n')[1].split(' ')[0]))
        except NoSuchElementException:
            bus_walk.append(None) 
    
    # parking:
        try:
            child= driver.find_element_by_xpath("//span[@class='mdi mdi-car mdi-36px noprint']")
            parent= child.find_element_by_xpath("..")
            parking.append(1)
        except NoSuchElementException:
            parking.append(0)
            


            

driver.quit()
   
    
        

In [28]:
df = pd.DataFrame(data = {'size':size,'rooms':numOfRooms,'price':price,'floor':floor,'kitchen':hasKitchen,
                          'heating Type':heatingType,'longitude':longitude,'latitude':latitude,'WashingMach':hasWashingMachine,
                          'Balcony':hasBalcony,'Elevetor':hasElevetor,'bus_walking':bus_walk,'parking':parking,'url':apartment_url})
df

<ipython-input-28-c298516cc8cc>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['floor'][i] = a[df['floor'][i]]


,size,rooms,price,floor,kitchen,heating Type,longitude,latitude,WashingMach,Balcony,Elevetor,bus_walking,parking,url
0,65m²,2,1200€,1st floor,own kitchen,central heating,13.450663,52.508722,1,1,0,2.0,1,https://www.wg-gesucht.de/wohnungen-in-Berlin-...
1,50m²,2,55€,3rd floor,None,central heating,13.403059,52.498443,1,0,0,5.0,1,https://www.wg-gesucht.de/wohnungen-in-Berlin-...
2,65m²,2,850€,3rd floor,own kitchen,None,13.368383,52.549651,1,1,0,2.0,0,https://www.wg-gesucht.de/wohnungen-in-Berlin-...
3,85m²,3,1100€,3rd floor,None,gas heating,13.424201,52.532498,1,1,0,2.0,1,https://www.wg-gesucht.de/wohnungen-in-Berlin-...
4,90m²,4,900€,ground floor,own kitchen,gas heating,13.204019,52.548280,1,0,0,6.0,1,https://www.wg-gesucht.de/wohnungen-in-Berlin-...
5,50m²,"1,5",650€,None,own kitchen,None,13.384552,52.538283,0,1,0,NaN,0,https://www.wg-gesucht.de/wohnungen-in-Berlin-...
6,53m²,2,1200€,4th floor,None,central heating,13.423312,52.548025,0,0,0,2.0,1,https://www.wg-gesucht.de/wohnungen-in-Berlin-...
7,52m²,2,25€,1st floor,None,gas heating,13.417041,52.536043,1,1,0,5.0,1,https://www.wg-gesucht.de/wohnungen-in-Berlin-...
8,56m²,2,500€,2nd Floor,None,None,13.328373,52.491616,1,1,1,1.0,1,https://www.wg-gesucht.de/wohnungen-in-Berlin-...
9,100m²,3,1000€,4th floor,own kitchen,district heating,13.421546,52.473161,1,1,0,5.0,1,https://www.wg-gesucht.de/wohnungen-in-Berlin-...


In [84]:
df.to_csv(r"C:\Users\Kostya\Desktop\Apartments\apartments1.csv")

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   size          26 non-null     object 
 1   rooms         26 non-null     object 
 2   price         26 non-null     object 
 3   floor         23 non-null     object 
 4   kitchen       14 non-null     object 
 5   heating Type  16 non-null     object 
 6   longitude     24 non-null     float64
 7   latitude      24 non-null     float64
 8   WashingMach   26 non-null     int64  
 9   Balcony       26 non-null     int64  
 10  Elevetor      26 non-null     int64  
 11  bus_walking   21 non-null     float64
 12  parking       26 non-null     int64  
 13  url           26 non-null     object 
dtypes: float64(3), int64(4), object(7)
memory usage: 3.0+ KB
